<a href="https://colab.research.google.com/github/HadarMiriamIsaacson/BS-SE-24-207/blob/main/How_to_run_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

enter the following link and save the folder in it on your gdrive to run it locally:
 https://drive.google.com/drive/folders/1s7ffc20Z2Ec6X4haU-MNOAsa87Ts7sle
 save the folder path string (path_to_local_model_on_my_gdrive). run the code provided below. you may preform changes only on the "text" variable content.

In [ ]:
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizer

# Load the tokenizer and model from the saved path
path_to_model_folder = 'path_to_local_model_on_my_gdrive'
tokenizer = RobertaTokenizer.from_pretrained(path_to_model_folder)
model = RobertaForTokenClassification.from_pretrained(path_to_model_folder)

# Move the model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example text data to extract keywords from
text = """
C, C++, C#, Qradar, Data analysis, SQL, Hunting cyber activity, Help desk,
Java Script, NetWitness, Active Directory, Java, McAfee ePolicy Server, Troubleshooting,
Python, Elastic search, Intsights, Machine Learning, Linux, MongoDB.
analyzed malicious files, techniques, and vulnerabilities to develop protections for Check Point products.
Released daily protection updates and continuously monitored threat detections.
Analyzed high-profile cyber-attacks and developed mitigation strategies.
Monitored sensor data to identify and manage potential targeted attacks.
Handled alerts from field reps and customers, providing immediate threat coverage.
Utilized Linux systems to implement and manage security protections.
Crane Operator at XYZ Construction
Operate and maintain various types of cranes including tower, mobile, and overhead cranes
Ensure safe and efficient crane operations on construction sites
Collaborate with construction team to ensure timely completion of projects
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

# Run inference to get predictions
with torch.no_grad():
    outputs = model(**inputs).logits

# Get the predictions by taking the argmax
predictions = torch.argmax(outputs, dim=2)

# Map predictions to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
predicted_labels = predictions[0].cpu().tolist()

# Extract and clean up the extracted keywords
keywords = []
current_phrase = []
for token, label in zip(tokens, predicted_labels):
    if label == 1:  # Assuming 1 indicates a keyword
        clean_token = token.replace('Ġ', '').strip()  # Remove leading space indicator and trim spaces
        if clean_token and clean_token not in ['<s>', '</s>']:
            current_phrase.append(clean_token)
    else:
        if current_phrase:
            # Join tokens to form the complete keyword phrase
            keywords.append(' '.join(current_phrase))
            current_phrase = []

# Append the last phrase if it was part of a keyword
if current_phrase:
    keywords.append(' '.join(current_phrase))

# Further clean up keywords by splitting overly long strings and handling punctuation
final_keywords = []
for kw in keywords:
    # Split by common delimiters and add each as a separate keyword
    for phrase in kw.split(','):
        phrase = phrase.strip()
        if phrase and phrase.isascii():
            final_keywords.append(phrase)

# Print the cleaned keywords
print("Extracted Keywords:", final_keywords)


Extracted Keywords: ['C ++', 'C #', 'Q rad ar', 'Data analysis', 'SQL', 'Hunting cyber activity', 'Help desk', 'Net Witness', 'Active Directory', 'Java', 'Mc Afee e Policy Server', 'Trou bles hooting', 'Elastic search', 'Int s ights', 'Machine Learning', 'Linux', 'techniques', 'mobile']
